# RTM Example
We will perform RTM using the following steps:
1. Read the 10m resampled models output from the FWI notebook
    * true model
    * initial model
    * fwi model
2. Visualize the models 
3. Build a small local compute cluster (2 workers)
4. Create list of shot locations 
5. Build F, the distributed block operator for seismic modeling
6. Use F and generate "observed data" in the true model 
7. Apply simple direct arrival mute 
8. Use the Jacobian of F to migrate the observed data in each model
9. Perform a little post migration filtering
10. Visualize Results

#### Note on runtime
This notebook takes approximately 1 hour to run for 24 shots with two workers on an Intel 8168.

`lscpu` CPU information: `Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz`

## Add required packages

In [ ]:
using Distributed, PyPlot, Jets, JetPack, JetPackDSP, Printf

## Record time for start of notebook

In [ ]:
time_beg = time()

## 1. Read the models output from the FWI notebook

In [ ]:
file1 = "../50_fwi/marmousi_resampled_10m_349x1701_vtrue.bin"
file2 = "../50_fwi/marmousi_resampled_10m_349x1701_vinit.bin"
filef = "../50_fwi/marmousi_resampled_10m_349x1701_vfwi.bin"

nz,nx = 349,1701
dz,dx = 10.0,10.0

v1_orig = read!(file1, Array{Float32}(undef,nz,nx));
v2_orig = read!(file2, Array{Float32}(undef,nz,nx));
vf_orig = read!(filef, Array{Float32}(undef,nz,nx));

####  Apply a 3x3 convolutional smoother
We perform the RTM migration in slightly smoothed models.

In [ ]:
ns = 21
P = JopPad(JetSpace(Float32,nz,nx), -ns:nz+ns, -ns:nx+ns, extend=true)
M = JopMix(range(P), (7,7))
R = JopPad(JetSpace(Float32,nz,nx), -ns:nz+ns, -ns:nx+ns, extend=false)

s1 = R' ∘ M ∘ P * (1 ./ v1_orig)
s2 = R' ∘ M ∘ P * (1 ./ v2_orig)
sf = R' ∘ M ∘ P * (1 ./ vf_orig)

v1 = 1 ./(s1);
v2 = 1 ./(s2);
vf = 1 ./(sf);

## 2. Visualize the models

In [ ]:
vmin,vmax = extrema(v1_orig)
@show vmin,vmax;

figure(figsize=(8,9)); clf()

subplot(3,2,1);imshow(v1_orig,aspect="auto",cmap="jet");
colorbar(orientation="horizontal");clim(vmin,vmax);
title("True Velocity")

subplot(3,2,2);imshow(v1,aspect="auto",cmap="jet");
colorbar(orientation="horizontal");clim(vmin,vmax);
title("True Velocity (smoothed)")


subplot(3,2,3);imshow(v2_orig,aspect="auto",cmap="jet");
colorbar(orientation="horizontal");clim(vmin,vmax);
title("Initial Velocity");

subplot(3,2,4);imshow(v2,aspect="auto",cmap="jet");
colorbar(orientation="horizontal");clim(vmin,vmax);
title("Initial Velocity (smoothed)");


subplot(3,2,5);imshow(vf_orig,aspect="auto",cmap="jet");
colorbar(orientation="horizontal");clim(vmin,vmax);
title("FWI Velocity");

subplot(3,2,6);imshow(vf,aspect="auto",cmap="jet");
colorbar(orientation="horizontal");clim(vmin,vmax);
title("FWI Velocity (smoothed)");

tight_layout()

## 3. Build a small local compute cluster (2 workers) 

#### Setup OMP environment variables for the cluster

In the distributed compute case the workers that we add would be on different hardware, and we might add tens of workers in 2D and hundreds in 3D. Here we run on a single machine with only 2 workers, and so we need to be careful with details related to high performance computing. If we did not specify thread affinity, the two workers would compete for the same physical cores and the modeling would be *incredibly* slow.

We spin up the small 2-worker cluster by calling `addprocs(2)`, and because we set the environment variable `ENV["OMP_DISPLAY_ENV"] = "true"` we will see the OMP environment printed out on each worker. In that output (below) we can verify that half of the total threads (44/2 = 22) are assigned to each socket on this 2 socket system. You can obtain more details about the hardware with the shell command `lscpu`.

We set four environment variables related to OpenMP:
* `OMP_DISPLAY_ENV` prints out the OpenMP environment on each worker
* `OMP_PROC_BIND` specifies that threads should be bound to physical cores
* `OMP_NUM_THREADS` specifies the number of threads per workers is 1/2 the number of physical cores
* `GOMP_CPU_AFFINITY` specifies which physical cores the threads run on for each worker

If you run the shell command `top` during execution, you will see 3 julia processes: the main process and two workers. The two workers should generally have about 50% of the system, and `load average` should tend towards the physical number of cores.

In [ ]:
nthread = Sys.CPU_THREADS
ENV["OMP_DISPLAY_ENV"] = "true"
ENV["OMP_PROC_BIND"] = "close"
ENV["OMP_NUM_THREADS"] = "$(div(nthread,2))" 
addprocs(2)
@show workers()
for k in 1:nworkers()
    place1 = (k - 1) * div(nthread,nworkers())
    place2 = (k + 0) * div(nthread,nworkers()) - 1
    @show place1, place2, nthread
    @spawnat workers()[k] ENV["GOMP_CPU_AFFINITY"] = "$(place1)-$(place2)";
end

In [ ]:
@everywhere using DistributedArrays, DistributedJets, Jets, JetPack, WaveFD, JetPackWaveFD, Random, LinearAlgebra

## 4. Create list of shot locations 
We use 100 shot locations, many times than our FWI example, and run at significantly higher frequency. 

In [ ]:
nshots = 100
sx = round.(Int,collect(range(0,stop=(nx-1)*dx,length=nshots)))
@show nshots;
@show sx;

## 5. Build F, the distributed block operator for seismic modeling

#### Note on scratch space for temporary files
When dealing with serialized nonlinear wavefields as in this example, we need to specify the location where scratch files will be written.

You may need to change this to point to a temporary directory available on your system.

In [ ]:
@everywhere scratch = "/mnt/scratch"
@assert isdir(scratch)

In [ ]:
# notice we model to significantly shorter time than for FWI
@everywhere begin
    ntrec = 3001
    dtrec = 0.002
    dtmod = 0.001
end

@everywhere function makeF(i,nz,nx,dz,dx,sx)
    JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        ntrec = ntrec,
        dtrec = dtrec,
        dtmod = dtmod,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=10.0),
        sx = sx[i],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        nbz_cache = nz,
        nbx_cache = 16,
        comptype = UInt32,
        srcfieldfile = joinpath(scratch, "field-$i-$(randstring()).bin"),
        reportinterval=12001)
end

In [ ]:
F = @blockop DArray(I->[makeF(i,nz,nx,dz,dx,sx) for i in I[1], j in I[2]], (nshots,1))

## 6. Use F and generate "observed data" in the true model 

In [ ]:
t1 = @elapsed begin
    d1 = F*v1
end
@show t1;

In [ ]:
@printf("Time for modeling %.2f minutes\n", t1 / 60)

## 7. Apply simple direct arrival mute 
We apply a simple mute based on the Marmousi water velocity of 1500 m/s and the offset from the traces headers. 

#### Define two functions to implement the mute for distributed block arrays
* The 1st function dispatches on the remote worker that "owns" the block of data
* The 2nd function implements a simple step function mute. 

In [ ]:
@everywhere function timemute!(F, d::DBArray, watervel, tmute)
    @sync for pid in procs(d)
        @async remotecall_fetch(timemute_local!, pid, F, d, watervel, tmute)
    end
end

@everywhere function timemute_local!(F, d::DBArray{T}, watervel, tmute) where {T}
    for iblock in localblockindices(d)
        dᵢ = getblock(d, iblock)
        Fᵢ = getblock(F, iblock, 1)
        nt = state(Fᵢ, :ntrec)
        for i = 1:length(state(Fᵢ, :rx)) 
            rx = state(Fᵢ, :rx)
            rz = state(Fᵢ, :rz)
            sx = state(Fᵢ, :sx)
            sz = state(Fᵢ, :sz)
            dist = sqrt((sx[1] - rx[i])^2 + (sz[1] - rz[i])^2)
            time = dist / watervel
            tbeg = 1
            tend = min(nt,round(Int, (time + tmute) / state(Fᵢ,:dtrec)))
            dᵢ[tbeg:tend,i] .= 0
        end
    end
    nothing
end

#### Apply the mute and visualize the results for 2 shot gathers 
The mute time chosen is six periods (at Ricker peak frequency) beyond water arrival. Six periods was empirically tested to remove the refraction energy and improve the reflection image. 

In [ ]:
watervel = minimum(v1)
tmute = 6/20
@show watervel
@show tmute

# copy distributed block array
d1_mute = zeros(range(F));
d1_mute .= d1;

# apply mute
timemute!(F, d1_mute, watervel, tmute)

# get local copies of 1st shot gather to plot
shots = [1, div(nshots,2), nshots]
local_d1 = [getblock(d1,k) for k in shots];
local_d1_mute = [getblock(d1_mute,k) for k in shots];

In [ ]:
scale = 20.0 / sqrt(norm(d1)^2 / length(d1))
@show scale

figure(figsize=(8,6)); clf()

subplot(2,2,1);
imshow(scale .* local_d1[1],cmap="gray",aspect="auto",clim=[-1,+1]);
title("Unmuted data for shot 1");

subplot(2,2,2);
imshow(scale .* local_d1_mute[1],cmap="gray",aspect="auto",clim=[-1,+1]);
title("Muted data for shot 1");

subplot(2,2,3);
imshow(scale .* local_d1[end],cmap="gray",aspect="auto",clim=[-1,+1]);
title("Unmuted data for shot $(nshots)");

subplot(2,2,4);
imshow(scale .* local_d1_mute[end],cmap="gray",aspect="auto",clim=[-1,+1]);
title("Muted data for shot $(nshots)");

tight_layout();

## 8. Use the Jacobian of F to migrate the observed data in each model
Note there is no water bottom mute or preprocessing applied here at all, those would clearly be improvements to this brute force workflow.

#### Migrate in the true velocity model

In [ ]:
t1 = @elapsed begin
    J1 = jacobian!(F, v1)
    illum1 = srcillum(J1)
    m1 = J1'*d1_mute;
end
@show extrema(m1)
@show extrema(illum1)

In [ ]:
@printf("Migrate time true model; %12.4f minutes\n", t1 / 60)

#### Migrate in the initial velocity model

In [ ]:
t2 = @elapsed begin
    J2 = jacobian!(F, v2)
    illum2 = srcillum(J2)
    m2 = J2'*d1_mute;
end
@show extrema(m2)
@show extrema(illum2)

In [ ]:
@printf("Migrate time init model; %12.4f minutes\n", t2 / 60)

#### Migrate in the FWI velocity model

In [ ]:
tf = @elapsed begin
    Jf = jacobian!(F, vf)
    illumf = srcillum(Jf)
    mf = Jf'*d1_mute;
end
@show extrema(mf)
@show extrema(illumf)

In [ ]:
@printf("Migrate time FWI  model; %12.4f minutes\n", tf / 60)

## 9. Perform a little post migration filtering

#### Laplacian filter to remove backscattered noise

In [ ]:
L = JopLaplacian(JetSpace(Float32,nz,nx))
# L = JopHighpass(JetSpace(Float32,nz,nx))

#### Apply low cut filter, illumination compensation, and gain

In [ ]:
g = ([0:(nz-1);]*dz).^2 * ones(1,nx);

# img1 = g .* (L * m1)
# img2 = g .* (L * m2)
# imgf = g .* (L * mf)

img1 = g .* (L * m1) ./ (illum1 .+ 1e-8 * maximum(abs, illum1));
img2 = g .* (L * m2) ./ (illum2 .+ 1e-8 * maximum(abs, illum2));
imgf = g .* (L * mf) ./ (illumf .+ 1e-8 * maximum(abs, illumf));

@show extrema(img1)
@show extrema(img2)
@show extrema(imgf)

#### Apply water bottom mute

In [ ]:
img1[v1.==1500.0] .= 0;
img2[v1.==1500.0] .= 0;
imgf[v1.==1500.0] .= 0;

## 10. Visualize Results

In [ ]:
mrms1 = 2.5 * sqrt(norm(img1)^2 / length(img1))
mrms2 = 2.5 * sqrt(norm(img2)^2 / length(img2))
mrmsf = 2.5 * sqrt(norm(imgf)^2 / length(imgf))
@show mrms1
@show mrms2
@show mrmsf

figure(figsize=(8,9)); clf()

subplot(3,1,1);imshow(img2,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-mrms2,+mrms2);
title("Migration in Initial Velocity");

subplot(3,1,2);imshow(img1,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-mrms1,+mrms1);
title("Migration in True Velocity")

subplot(3,1,3);imshow(imgf,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-mrmsf,+mrmsf);
title("Migration in FWI Velocity");

tight_layout()

## Remove local files and workers

In [ ]:
close(F);
rmprocs(workers());

## Record time at end of notebook
Report total elapsed time to run the notebook

In [ ]:
time_end = time()
@sprintf("Time to run notebook; %.2f minutes\n", (time_end - time_beg) / 60)